In [1]:
import pandas as pd
import duckdb as db
from duckdb import ConstraintException
from datetime import datetime
import os
from tqdm import tqdm
from duckdb import CatalogException

# We're using the google style guide for python (https://google.github.io/styleguide/pyguide.html)

In [2]:
# establish a connection to the database
db_path = '../data/database/patent_database'
con = db.connect(database=db_path, read_only=False)

In [ ]:
# Read the data from the CSV file
firm_names_data = pd.read_stata('../../files_maris/katharina_patents_name_matching/all_names_incl_isin.dta').reset_index()

In [4]:
con.sql("show tables")

┌────────────────────┐
│        name        │
│      varchar       │
├────────────────────┤
│ firm_data_complete │
│ firm_isin          │
│ firm_names         │
│ patstat_data       │
│ patstat_firm_match │
└────────────────────┘

In [4]:
def write_log(msg: str, logfile: str):
    """Writes a message to the log file.

    Args:
        msg: The message to write to the log file
        logfile: The name of the log file

    Returns:
        None
    """
    file_path = f"../logs/{logfile}"
    with open(file_path, "a") as log_file:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"{timestamp}\n{msg}\n\n")

def wipe_database():
    """Deletes all tables in the database.

    Args:
        None

    Returns:
        None
    """
    # Get names of all tables
    tables = con.sql("SHOW TABLES").fetchdf().name.to_list()
    while len(tables) > 0:
        for table in tables:
            try:
                con.sql(f"DROP TABLE {table}")
                tables.remove(table)
            except CatalogException as e:
                # Since we have foreign keys, we might get exceptions,
                # when deleting in wrong order.
                print(repr(e))
                continue

In [5]:
def clean_firm_isin_table():
    """Gets rid of some duplicate entries in the firm_isin table. This is necessary since some firms have multiple ISINs. We only keep one ISIN per firm.

    Args:
        None

    Returns:
        None
    """
    sql = """
    DELETE FROM firm_isin
    WHERE ISIN = ?;
    """
    con.execute(sql, ('US1746132083',)) # CITIZEN FINANCIAL CORP
    con.execute(sql, ('JP3360850006',)) # SHOEI CO LTD
    con.execute(sql, ('JP3386660009',)) # JTECH CORP
    con.execute(sql, ('US70978T1079',)) # PEOPLES BANCORP INC
    con.execute(sql, ('JP3384710004',)) # SHINWA CO LTD


def create_base_tables(firm_names_data: pd.DataFrame, reset: bool = False):
    """Creates the base tables for the name matching process. This includes the firm_names_data table and the patstat_data table. Also create firm_data_complete table.

    Args:
        reset: If True, the tables will be dropped if they already exist.
        firm_names_data: The data containing the names and ISINs of the firms.

    Returns:
        None
    """
    if reset:
        # -- 1. Drop old tables if they exist
        con.sql("DROP TABLE IF EXISTS firm_isin;")
        con.sql("DROP TABLE IF EXISTS firm_names;")
        con.sql("DROP TABLE IF EXISTS firm_data_complete;")

    # -- 2. Create firm_names table with a generated surrogate key for each distinct name
    # We do this to get rid of duplicates for name matching
    sql = """
        CREATE TABLE IF NOT EXISTS firm_names (
            firm_id INTEGER PRIMARY KEY,
            name VARCHAR(255) NOT NULL
        );
    """
    con.sql(sql)

    sql = """
        INSERT INTO firm_names (firm_id, name)
        SELECT ROW_NUMBER() OVER (ORDER BY name) AS firm_id,
               name
        FROM (
            SELECT DISTINCT UPPER(name) AS name
            FROM firm_names_data
        ) AS sub;
    """
    con.sql(sql)

    # -- 3. Create firm_isin table, referencing the firm_id via a JOIN on the name
    # We do this, since some firms have multiple ISINs (e.g. SHINWA CO LTD). This could be due to different classes of shares / or subsidiaries?
    sql = """
        CREATE TABLE IF NOT EXISTS firm_isin (
           ISIN VARCHAR(255) PRIMARY KEY NOT NULL,
           firm_id INTEGER REFERENCES firm_names(firm_id)
        );
    """
    con.sql(sql)

    sql = """
        INSERT INTO firm_isin (ISIN, firm_id)
        SELECT i.ISIN, f.firm_id
        FROM (
           SELECT DISTINCT ISIN, UPPER(name) AS name
           FROM firm_names_data
        ) AS i
        JOIN firm_names AS f
        ON i.name = f.name;
    """
    con.sql(sql)
    clean_firm_isin_table();

    # -- 4. This table can be used to extract the bdvids of the subsidiaries via ISIN or name
    con.sql("CREATE TABLE IF NOT EXISTS firm_data_complete AS SELECT ISIN, subsidiarybvdid, UPPER(name) AS name FROM firm_names_data")

def create_match_table(reset: bool = False):
    """Creates the patstat_firm_match table in the database. This table is used to store the matches between the patstat data and the firm (id) data.

    Args:
        reset: If True, the table will be dropped if it already exists.

    Returns:
        None
    """
    if reset:
        con.sql("DROP TABLE IF EXISTS patstat_firm_match;")

    sql = """
    CREATE TABLE IF NOT EXISTS patstat_firm_match (
        han_id INTEGER PRIMARY KEY, -- han_id from patstat_data tables
        firm_id INTEGER REFERENCES firm_names(firm_id), -- firm_id from firm_names table
        similarity FLOAT, -- similarity between the patstat name and the firm name
    );
    """
    con.sql(sql)

def create_patstat_database(reset: bool = True):
    """Creates a table in the database with the patstat data.

    Args:
        reset: If True, the table will be dropped if it already exists.

    Returns:
        None
    """
    if reset:
        con.sql(f"DROP TABLE IF EXISTS patstat_data")

    sql = f"""
        CREATE TABLE IF NOT EXISTS patstat_data (
            pat_publn_id INTEGER,
            publn_date DATE,
            han_id INTEGER,
            han_name VARCHAR(255),
            person_name VARCHAR(255),
            psn_name VARCHAR(255),
        );
    """
    con.execute(sql)

def insert_patstat_data(patstat_data: pd.DataFrame):
    """Inserts the patstat data into the database.

    Args:
        patstat_data: The data containing the patstat data.

    Returns:
        None
    """
    con.register('patstat_data_tmp', patstat_data)
    sql = """
        INSERT INTO patstat_data
        SELECT DISTINCT pat_publn_id, publn_date, han_id, 
                        UPPER(han_name) AS han_name, 
                        UPPER(person_name) AS person_name, 
                        UPPER(psn_name) AS psn_name
        FROM patstat_data_tmp;
    """
    con.execute(sql)
    con.execute("DROP VIEW IF EXISTS patstat_data_tmp")




In [6]:
def insert_match(han_id: int, firm_id: int, similarity: float):
    """Inserts a match between a patstat publication id and a firm id into the patstat_firm_match table.

    Args:
        han_id: The patstat firm / patent applicant id.
        firm_id: The firm id.
        similarity: The similarity between the patstat name and the firm name.

    Returns:
        None
    """
    # Get current matching firm / score
    match = con.execute(f"SELECT * FROM patstat_firm_match WHERE han_id = {han_id}").fetchdf()
    # If we already have a match with a higher similarity, we don't insert the new match
    if len(match) > 0:
        current_similarity = match['similarity'].iloc[0]
        if current_similarity is None or similarity is None:
            pass
        elif current_similarity > similarity:
            return
    # Convert values to Python types that DuckDB supports natively
    han_id = int(han_id) if han_id is not None else None
    firm_id = int(firm_id) if firm_id is not None else None
    similarity = float(similarity) if similarity is not None else None
    # print(f"Inserting match for pat_publn_id {pat_publn_id}, firm_id {firm_id}, similarity {similarity}, pat_table {pat_table}")
    try:
        sql = f"""
            INSERT INTO patstat_firm_match (han_id, firm_id, similarity)
            VALUES (?, ?, ?);
        """
        con.execute(sql, (han_id, firm_id, similarity))
    except ConstraintException as e:
        error = f"{repr(e)}\nError inserting match for pat_publn_id firm_id {firm_id}, similarity {similarity}"
        write_log(error, "insert_match_error.log")


def get_matching_firm(patstat_name: str) -> pd.DataFrame:
    """Gets the best matching firm for a given patstat name. Jayo-Winkler similarity is used to calculate the similarity between the patstat name and the firm name.

    """
    patstat_name = patstat_name.replace("'", "")
    # We use the Jaro-Winkler similarity since it is a good metric for string similarity. It
    sql = f"""
        SELECT
            firm_id,
            name,
            jaro_winkler_similarity(name, '{patstat_name}') AS similarity
        FROM firm_names
        WHERE similarity > 0.8
        ORDER BY similarity DESC
        LIMIT 1;
    """
    df = con.sql(sql).fetchdf()
    return df


In [7]:
def extract_before_first_comma(s: str) -> str:
    """Extracts the substring before the first comma in a string.

    Args:
        s: The input string.

    Returns:
        The substring before the first comma.
    """
    # @todo check if this results in errors
    if s is None:
        return ""
    # Split on the first comma only (maxsplit=1)
    parts = s.split(',', 1)
    return parts[0]  # If there's no comma, split() returns [s]

def get_best_match(patstat_entry: pd.Series) -> pd.DataFrame:
    """Calculates the best name match for a patstat entry against our firm_names table.

    Args:
        patstat_entry: A single entry from the patstat data.

    Returns:
        The best match for the patstat entry.
    """

    df = None
    max_similarity = -1

    # We have to use the extract_before_first_comma function to get rid of the location information in the name
    df_han = get_matching_firm(extract_before_first_comma(patstat_entry['han_name']))
    df_psn = get_matching_firm(extract_before_first_comma(patstat_entry['psn_name']))
    df_person = get_matching_firm(extract_before_first_comma(patstat_entry['person_name']))
    # We check if we have a match with a similarity > 0.9 in any of the dataframes. They are sorted by the subjective importance (han_name > psn_name > person_name)
    # If we have a match with a similarity > 0.9, we instantly return it
    if len(df_han) > 0:
        similarity = df_han['similarity'].iloc[0]
        if similarity > 0.9:
            return df_han
        elif similarity > max_similarity:
            max_similarity = similarity
            df = df_han
    if len(df_psn) > 0:
        similarity = df_psn['similarity'].iloc[0]
        if similarity > 0.9:
            return df_psn
        elif similarity > max_similarity:
            max_similarity = similarity
            df = df_psn
    if len(df_person) > 0:
        similarity = df_person['similarity'].iloc[0]
        if similarity > 0.9:
            return df_person
        elif similarity > max_similarity:
            max_similarity = similarity
            df = df_person

    # If we don't have a match with a similarity > 0.9, we return the best match
    return df


def process_patstat_entry(patstat_entry: pd.Series):
    """Processes a single entry from the patstat data and tries to find a matching firm.

    Args:
        patstat_entry: A single entry from the patstat data.

    Returns:
        None
    """
    data = get_best_match(patstat_entry)

    if data is not None:
        data = data.iloc[0]
        insert_match(patstat_entry['han_id'], data['firm_id'], data['similarity'])
    else:
        insert_match(patstat_entry['han_id'], None, None)

In [8]:
def process_patstat_files():
    """Processes the patstat_data and tries to find matching firms for each entry.

    Args:
        None
    Returns:
        None
    """

    sql = """
        SELECT han_id, han_name, person_name, psn_name
        FROM patstat_data
        WHERE han_id IN (SELECT DISTINCT(han_id) FROM patstat_data)

    """
    data = con.execute(sql).fetchdf()
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing rows"):
        process_patstat_entry(row)


In [9]:
def load_patstat_data():
    """" Load given patstat data into the database.

    Args:
        None

    Returns:
        None
    """
    base_path = '../data/patstat_data/patent_download_sep_2025/Teil2'
    # Get all files in the directory
    files = os.listdir(base_path)
    for file in files:
        if file.endswith(".csv"):
            path = f'{base_path}/{file}'
            # print(f"Processing file {file}. Path: {base_path}")
            patstat_data = pd.read_csv(path, sep=';')
            insert_patstat_data(patstat_data)

In [11]:
if __name__ == "__main__":
    # print("Wiping database")
    # wipe_database()
    # print("Loading patstat data")
    # create_base_tables(firm_names_data)
    # create_patstat_database()
    # create_match_table()
    load_patstat_data()
    print("Start processing")
    # process_patstat_files()

/scratch/slurm_tmpdir/job_1395162/ipykernel_55153/519815376.py:17: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  patstat_data = pd.read_csv(path, sep=';')
/scratch/slurm_tmpdir/job_1395162/ipykernel_55153/519815376.py:17: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  patstat_data = pd.read_csv(path, sep=';')
/scratch/slurm_tmpdir/job_1395162/ipykernel_55153/519815376.py:17: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  patstat_data = pd.read_csv(path, sep=';')
/scratch/slurm_tmpdir/job_1395162/ipykernel_55153/519815376.py:17: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  patstat_data = pd.read_csv(path, sep=';')
/scratch/slurm_tmpdir/job_1395162/ipykernel_55153/519815376.py:17: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set lo

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Start processing


In [105]:
get_matching_firm("")

,firm_id,name,similarity


In [15]:
sql = """
    SELECT DISTINCT(firm_id), han_id, similarity, name, han_name, person_name, psn_name FROM patstat_firm_match
    JOIN firm_names USING(firm_id)
    JOIN patstat_data USING(han_id)
    WHERE similarity >= 0.9
    """
con.execute(sql).fetchdf()

,firm_id,han_id,similarity,name,han_name,person_name,psn_name
0,888,132992,1.000000,APPLIED MATERIALS INC,APPLIED MATERIALS INC,"APPLIED MATERIALS, INC.",APPLIED MATERIALS
1,3786,696112,1.000000,DISCO CORP,DISCO CORP,DISCO CORPORATION,DISCO CORPORATION
2,10592,1438789,0.910331,PANGEN BIOTECH INC,JANSSEN BIOTECH INC,"JANSSEN BIOTECH, INC.",JANSSEN BIOTECH
3,445,60901,1.000000,AJINOMOTO CO INC,AJINOMOTO CO INC,"AJINOMOTO CO., INC.",AJINOMOTO COMPANY
4,7900,2081711,1.000000,KUBOTA CORP,KUBOTA CORP,KUBOTA CORPORATION,KUBOTA CORPORATION
...,...,...,...,...,...,...,...
30663,6610,1656582,1.000000,IIDA GROUP HOLDINGS CO LTD,IIDA GROUP HOLDINGS CO LTD,"IIDA GROUP HOLDINGS CO., LTD.",IIDA GROUP HOLDINGS COMPANY
30664,12622,165728428,0.940000,SIEMENS AG,СИМЕНС АКЦИЕНГЕЗЕЛЬШАФТ,СИМЕНС АКЦИЕНГЕЗЕЛЬШАФТ,SIEMENS
30665,12236,161808648,0.903258,SCANDINAVIAN TOBACCO GROUP A/S,СКАНДИНАВИАН ИННОВЭЙШН ГРУП ОЙ,СКАНДИНАВИАН ИННОВЭЙШН ГРУП ОЙ,SCANDINAVIAN INNOVATION GROUP
30666,4458,3900421,0.977778,ENVIRONMENTAL CLEAN TECHNOLOGIES LTD,ENVAL CLEAN TECH LTD,ENVIRONMENTAL CLEAN TECHNOLOGIES LIMITED,ENVIRONMENTAL CLEAN TECHNOLOGIES LIMITED


In [16]:
sql = """
        SELECT count(*) FROM patstat_firm_match
    """

con.execute(sql).fetchdf()

,count_star()
0,718668


In [69]:
con.execute("SELECT * FROM patstat_data_tmp").fetchdf()

InvalidInputException: Invalid Input Error: Failed to cast value: Unimplemented type for cast (VARCHAR -> NULL)

In [ ]:
firm_names_data

In [45]:
buggy_data.loc[1]

Unnamed: 0                                    102
pat_publn_id                            540273784
firm_id                                     10836
similarity                                    1.0
pat_table                               patents_P
name                                   PFIZER INC
han_name               BOSTON MEDICAL CENTER CORP
person_name     BOSTON MEDICAL CENTER CORPORATION
psn_name        BOSTON MEDICAL CENTER CORPORATION
Name: 1, dtype: object

In [44]:
get_best_match(buggy_data.loc[1])

,firm_id,name,similarity
0,2008,BOSTON BEER COMPANY INC,0.877759
1,2010,BOSTON SCIENTIFIC CORP,0.857653
2,2009,BOSTON PROPERTIES INC,0.825458
3,2022,BOWLIN TRAVEL CENTERS INC,0.808065
4,15154,WHEATON PRECIOUS METALS CORP,0.801532


In [78]:
con.sql('ROLLBACK;')
con.sql('show tables')

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [97]:
con.sql('select * from patstat_patents_P').fetchdf()

,pat_publn_id,han_name,person_name,psn_name,han_id
0,306252117,RIFAT GJOTA,RIFAT GJOTA,RIFAT GJOTA,114348731
1,306479717,SERVOTROL SISTEMAS DE COMANDO AUTOMATICO LDA,SERVOTROL SISTEMAS DE COMANDO AUTOMATICO LDA,SERVOTROL - SISTEMAS DE COMANDO AUTOMATICO,134122706
2,306479722,GYORGY BARA,GYORGY BARA,GYORGY BARA,134122710
3,306479722,JOZSEF TALPAG,JOZSEF TALPAG,JOZSEF TALPAG,134122712
4,306479722,PAL GAVAJDA,PAL GAVAJDA,PAL GAVAJDA,134122711
...,...,...,...,...,...
113724,606189813,KITE PHARMA INC,"KITE PHARMA, INC.",KITE PHARMA,1968026
113725,606189855,LUKOMSKI TOMASZ,ŁUKOMSKI TOMASZ,ŁUKOMSKI TOMASZ,4557906
113726,606189859,ALFA LAVAL CORPORATE AB,ALFA LAVAL CORPORATE AB,ALFA LAVAL,92321
113727,606189861,ISTA SE,ISTA SE,ISTA SE,191743558


In [12]:
con.close()